<a href="https://colab.research.google.com/github/jayakedia10/Hangman/blob/main/Hangman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests
import random
import string
import time
import re
import collections
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()

        self.current_dictionary = []
        self.vectorizer = CountVectorizer(analyzer='char', ngram_range=(1, 3))
        self.classifier = MultinomialNB()
        self.train_model()

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']
        data = {link: 0 for link in links}
        for link in links:
            requests.get(link)
            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s
        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def train_model(self):
        X = self.vectorizer.fit_transform(self.full_dictionary)
        y = [word[0] for word in self.full_dictionary]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        self.classifier.fit(X_train, y_train)
        print(f"Model accuracy: {self.classifier.score(X_test, y_test)}")

    def guess(self, word):
            clean_word = word[::2].replace("_", ".")
            len_word = len(clean_word)

            self.current_dictionary = [w for w in self.full_dictionary if len(w) == len_word and re.match(clean_word, w)]

            if not self.current_dictionary:
                self.current_dictionary = self.full_dictionary

            pattern_features = self.vectorizer.transform([clean_word])
            letter_probs = self.classifier.predict_proba(pattern_features)[0]

            letter_freq = collections.Counter("".join(self.current_dictionary))
            combined_scores = {}

            for letter, prob in zip(self.classifier.classes_, letter_probs):
                if letter not in self.guessed_letters:
                    freq = letter_freq.get(letter, 0)
                    combined_scores[letter] = prob * (freq + 1)

            for i, char in enumerate(clean_word):
                if char == '.':
                    pos_freq = collections.Counter(word[i] for word in self.current_dictionary if i < len(word))
                    for letter, freq in pos_freq.items():
                        if letter not in self.guessed_letters:
                            combined_scores[letter] = combined_scores.get(letter, 0) + freq * 2

            if combined_scores:
                return max(combined_scores, key=combined_scores.get)

            for letter, _ in self.full_dictionary_common_letter_sorted:
                if letter not in self.guessed_letters:
                    return letter

            return random.choice(string.ascii_lowercase)

    def build_dictionary(self, dictionary_file_location):
        with open(dictionary_file_location, "r") as text_file:
            return text_file.read().splitlines()

    def start_game(self, practice=True, verbose=True):
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice": practice})
        if response.get('status') == "approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print(f"Successfully start a new game! Game ID: {game_id}. # of tries remaining: {tries_remains}. Word: {word}.")
            while tries_remains > 0:
                guess_letter = self.guess(word)
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print(f"Guessing letter: {guess_letter}")

                try:
                    res = self.request("/guess_letter", {"request": "guess_letter", "game_id": game_id, "letter": guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print(f"Server response: {res}")
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status == "success":
                    if verbose:
                        print(f"Successfully finished game: {game_id}")
                    return True
                elif status == "failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print(f"Failed game: {game_id}. Because of: {reason}")
                    return False
                elif status == "ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status == "success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        if self.access_token:
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# Main execution
if __name__ == "__main__":
    api = HangmanAPI(access_token="554919aecfc2a54b36199b6ec58084", timeout=2000)

    # Play practice games
    num_practice_games = 100
    successes = 0
    for i in range(num_practice_games):
        print(f'Playing practice game {i + 1}/{num_practice_games}')
        if api.start_game(practice=True, verbose=False):
            successes += 1
        time.sleep(0.5)  # To avoid overwhelming the server

    practice_success_rate = successes / num_practice_games
    print(f'Practice success rate: {practice_success_rate:.2%}')

    for i in range(1000):
        print(f'Playing recorded game {i + 1}/1000')
        api.start_game(practice=False, verbose=False)
        time.sleep(0.5)  # To avoid overwhelming the server

    [total_practice_runs, total_recorded_runs, total_recorded_successes, total_practice_successes] = api.my_status()
    print(f'Total practice runs: {total_practice_runs}')
    print(f'Total recorded runs: {total_recorded_runs}')


Model accuracy: 0.5688737351517817
Playing practice game 1/100
Playing practice game 2/100
Playing practice game 3/100
Playing practice game 4/100
Playing practice game 5/100
Playing practice game 6/100
Playing practice game 7/100
Playing practice game 8/100
Playing practice game 9/100
Playing practice game 10/100
Playing practice game 11/100
Playing practice game 12/100
Playing practice game 13/100
Playing practice game 14/100
Playing practice game 15/100
Playing practice game 16/100
Playing practice game 17/100
Playing practice game 18/100
Playing practice game 19/100
Playing practice game 20/100
Playing practice game 21/100
Playing practice game 22/100
Playing practice game 23/100
Playing practice game 24/100
Playing practice game 25/100
Playing practice game 26/100
Playing practice game 27/100
Playing practice game 28/100
Playing practice game 29/100
Playing practice game 30/100
Playing practice game 31/100
Playing practice game 32/100
Playing practice game 33/100
Playing practice 

HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}